In [250]:
import numpy as np
import pandas as pd
import os
import io
import re
from unidecode import unidecode

path = r"C:/Users/Avisia/Documents/Football Transfer Tool/"
os.chdir(path)
import warnings
warnings.filterwarnings("ignore")

In [251]:
players_stats = pd.read_csv('data/joueurs_avec_stats.csv')

## Création des styles de joueurs

In [252]:
players_stats.head()

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,poste_,poste,note_perf,nom_y,player_name,player_extended_name,date_of_birth,height,weight,perf
0,Alisson,1992-10-02,England,2020/2021,Liverpool,EPL,3,270,0,0,...,GK,GK,87.0,alisson,alisson,alisson ramses becker,1992-10-02,191.0,91.0,4.943182
1,Alisson,1992-10-02,England,2019/2020,Liverpool,EPL,29,2545,0,1,...,GK,GK,87.0,alisson,alisson,alisson ramses becker,1992-10-02,191.0,91.0,4.943182
2,Alisson,1992-10-02,England,2019/2020,Liverpool,UCL,5,450,0,0,...,GK,GK,87.0,alisson,alisson,alisson ramses becker,1992-10-02,191.0,91.0,4.943182
3,Alisson,1992-10-02,England,2019,Brazil,ICA,6,570,0,0,...,GK,GK,87.0,alisson,alisson,alisson ramses becker,1992-10-02,191.0,91.0,4.943182
4,Alisson,1992-10-02,England,2018/2019,Liverpool,EPL,38,3420,0,0,...,GK,GK,87.0,alisson,alisson,alisson ramses becker,1992-10-02,191.0,91.0,4.943182


In [253]:
cols = ['Player Name','Season', 'Tournament', 'Team','Apps', 'Mins', 'Goals', 'Assists', 'Goals/90min',
                    'Assists/90min', 'AerialWon','MotM', 'Rating']

In [254]:
p = ['Bukayo Saka', 'Eddie Nketiah']

In [255]:
a = players_stats[(players_stats['Player Name'].isin(p)) & (players_stats['Season']=='2018/2019')]

In [257]:
a

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,poste_,poste,note_perf,nom_y,player_name,player_extended_name,date_of_birth,height,weight,perf
3915,Eddie Nketiah,1999-05-30,England,2018/2019,Arsenal,EPL,0,63,1,0,...,ST,ATT,67.0,nketiah,nketiah,eddie nketiah,1999-05-30,175.0,70.0,3.602151
3916,Eddie Nketiah,1999-05-30,England,2018/2019,Arsenal,UEL,2,180,0,0,...,ST,ATT,67.0,nketiah,nketiah,eddie nketiah,1999-05-30,175.0,70.0,3.602151
3922,Bukayo Saka,2001-09-05,England,2018/2019,Arsenal,EPL,0,7,0,0,...,LM,ATT,67.0,saka,saka,bukayo saka,2001-09-05,178.0,70.0,3.602151
3923,Bukayo Saka,2001-09-05,England,2018/2019,Arsenal,UEL,1,112,0,0,...,LM,ATT,67.0,saka,saka,bukayo saka,2001-09-05,178.0,70.0,3.602151


In [258]:
l = ['UCL','UEL']

In [260]:
a[~a['Tournament'].isin(l)].set_index(['Player Name', 'Team'])

,,birth,nation,Season,Tournament,Apps,Mins,Goals,Assists,Yellow,Red,...,poste_,poste,note_perf,nom_y,player_name,player_extended_name,date_of_birth,height,weight,perf
Player Name,Team,,,,,,,,,,,,,,,,,,,,,
Eddie Nketiah,Arsenal,1999-05-30,England,2018/2019,EPL,0,63,1,0,0,0,...,ST,ATT,67.0,nketiah,nketiah,eddie nketiah,1999-05-30,175.0,70.0,3.602151
Bukayo Saka,Arsenal,2001-09-05,England,2018/2019,EPL,0,7,0,0,0,0,...,LM,ATT,67.0,saka,saka,bukayo saka,2001-09-05,178.0,70.0,3.602151


In [261]:
players_stats.shape

(15115, 151)

In [262]:
players_stats.reset_index(inplace=True, drop=True)

In [263]:
players_stats_clean=players_stats.loc[:,'Player Name':'id']

In [264]:
players_stats_clean.to_csv('data/joueurs_avec_stats_reelles.csv', index=False)

In [265]:
players = players_stats.drop_duplicates('Player Name')

In [266]:
players['style'] = players.apply(lambda x: [], axis=1)

for i in range(len(players)):
    rowIndex = players.index[i]
    # gardien
    if (players['gk_diving'].loc[rowIndex]>=75) & (players['gk_handling'].loc[rowIndex]>=75) & (players['gk_kicking'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('mur')
        players.loc[rowIndex,'style_mur'] = 1
    else : players.loc[rowIndex,'style_mur'] = 0
        
    if (players['gk_reflexes'].loc[rowIndex]>=75) & (players['gk_speed'].loc[rowIndex]>=75) & (players['gk_kicking'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('bouclier')
        players.loc[rowIndex,'style_bouclier'] = 1
    else : players.loc[rowIndex,'style_bouclier'] = 0
        
    if (players['gk_reflexes'].loc[rowIndex]>=75) & (players['gk_speed'].loc[rowIndex]>=75) & (players['gk_positioning'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('chat')
        players.loc[rowIndex,'style_chat'] = 1
    else : players.loc[rowIndex,'style_chat'] = 0
        
    if (players['gk_diving'].loc[rowIndex]>=75) & (players['gk_handling'].loc[rowIndex]>=75) & (players['gk_positioning'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('gant')
        players.loc[rowIndex,'style_gant'] = 1
    else : players.loc[rowIndex,'style_gant'] = 0
    
    # defenseur
    if (players['defending'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('sentinelle')
        players.loc[rowIndex,'style_sentinelle'] = 1
    else : players.loc[rowIndex,'style_sentinelle'] = 0
        
    if (players['defending'].loc[rowIndex]>=75) & (players['dribbling'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('garde')
        players.loc[rowIndex,'style_garde'] = 1
    else : players.loc[rowIndex,'style_garde'] = 0
        
    if (players['defending'].loc[rowIndex]>=75) & (players['shooting'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('gladiateur')
        players.loc[rowIndex,'style_gladiateur'] = 1
    else : players.loc[rowIndex,'style_gladiateur'] = 0
        
    if (players['defending'].loc[rowIndex]>=75) & (players['passing'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('colonne vertebrale')
        players.loc[rowIndex,'style_colonne_vertebrale'] = 1
    else : players.loc[rowIndex,'style_colonne_vertebrale'] = 0
        
    if (players['defending'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75) & (players['pace'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('pilier')
        players.loc[rowIndex,'style_pilier'] = 1
    else : players.loc[rowIndex,'style_pilier'] = 0
    
    if (players['defending'].loc[rowIndex]>=75) & (players['pace'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('ombre')
        players.loc[rowIndex,'style_ombre'] = 1
    else : players.loc[rowIndex,'style_ombre'] = 0
    
    # milieux de terrain    
    if (players['passing'].loc[rowIndex]>=75) & (players['dribbling'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('artiste')
        players.loc[rowIndex,'style_artiste'] = 1
    else : players.loc[rowIndex,'style_artiste'] = 0
    
    if (players['passing'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('architecte')
        players.loc[rowIndex,'style_architecte'] = 1
    else : players.loc[rowIndex,'style_architecte'] = 0
        
    if (players['passing'].loc[rowIndex]>=75) & (players['defending'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('roc')
        players.loc[rowIndex,'style_roc'] = 1
    else : players.loc[rowIndex,'style_roc'] = 0

    if (players['passing'].loc[rowIndex]>=75) & (players['shooting'].loc[rowIndex]>=75) & (players['dribbling'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('maestro')
        players.loc[rowIndex,'style_maestro'] = 1
    else : players.loc[rowIndex,'style_maestro']= 0
        
    if (players['passing'].loc[rowIndex]>=75) & (players['pace'].loc[rowIndex]>=75) & (players['dribbling'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('moteur')
        players.loc[rowIndex,'style_moteur'] = 1
    else : players.loc[rowIndex,'style_moteur'] = 0

    if (players['passing'].loc[rowIndex]>=75) & (players['pace'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('catalyseur')
        players.loc[rowIndex,'style_catalyseur'] = 1
    else : players.loc[rowIndex,'style_catalyseur'] = 0
        
    # attaquants
    if (players['shooting'].loc[rowIndex]>=75) & (players['dribbling'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append("tireur d'élite")
        players.loc[rowIndex,'style_tireur_delite'] = 1
    else : players.loc[rowIndex,'style_tireur_delite'] = 0
        
    if (players['shooting'].loc[rowIndex]>=75) & (players['pace'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('oeil de lynx')
        players.loc[rowIndex,'style_oeildelynx'] = 1
    else : players.loc[rowIndex,'style_oeildelynx'] = 0
        
    if (players['shooting'].loc[rowIndex]>=75) & (players['passing'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('faucon')
        players.loc[rowIndex,'style_faucon'] = 1
    else : players.loc[rowIndex,'style_faucon'] = 0
        
    if (players['shooting'].loc[rowIndex]>=75) & (players['dribbling'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('buteur')
        players.loc[rowIndex,'style_buteur'] = 1
    else : players.loc[rowIndex,'style_buteur'] = 0
        
    if (players['shooting'].loc[rowIndex]>=75) & (players['physic'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('finisseur')
        players.loc[rowIndex,'style_finisseur'] = 1
    else : players.loc[rowIndex,'style_finisseur'] = 0
        
    if (players['shooting'].loc[rowIndex]>=75) & (players['pace'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('chasseur')
        players.loc[rowIndex,'style_chasseur'] = 1
    else :players.loc[rowIndex,'style_chasseur'] = 0
    
    # traits spéciaux
    if (players['skill_fk_accuracy'].loc[rowIndex]>=75):
        players['style'].loc[rowIndex].append('tireur de coup franc')
        players.loc[rowIndex,'style_cf'] = 1
    else : players.loc[rowIndex,'style_cf'] = 0
        
    if (players['weak_foot'].loc[rowIndex]>=4):
        players['style'].loc[rowIndex].append('2 pieds')
        players.loc[rowIndex,'style_2pieds'] = 1
    else : players.loc[rowIndex,'style_2pieds'] = 0 
        
    if (players['skill_moves'].loc[rowIndex]==5):
        players['style'].loc[rowIndex].append('magicien')
        players.loc[rowIndex,'style_magicien'] = 1
    else : players.loc[rowIndex,'style_magicien'] = 0
    
    if (players['international_reputation'].loc[rowIndex]==5):
        players['style'].loc[rowIndex].append('légende')
        players.loc[rowIndex,'style_legende'] = 1
    else : players.loc[rowIndex,'style_legende'] = 0
        
    if (players['international_reputation'].loc[rowIndex]==4):
        players['style'].loc[rowIndex].append('star internationale')
        players.loc[rowIndex,'style_star'] = 1
    else : players.loc[rowIndex,'style_star'] = 0
        
        

In [267]:
players.head()

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,style_oeildelynx,style_faucon,style_buteur,style_finisseur,style_chasseur,style_cf,style_2pieds,style_magicien,style_legende,style_star
0,Alisson,1992-10-02,England,2020/2021,Liverpool,EPL,3,270,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,Fabinho,1993-10-23,England,2020/2021,Liverpool,EPL,4,391,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,Diogo Jota,1996-12-04,England,2020/2021,Liverpool,EPL,1,112,1,0,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
37,Thiago,1991-04-11,England,2020/2021,Liverpool,EPL,1,135,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
64,Ederson,1993-08-17,England,2020/2021,Man City,EPL,4,360,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Retrouver les bonnes photos

In [268]:
photos = pd.DataFrame({'id': players['id']})

In [269]:
photos['url_img'] = photos['id'].apply(lambda x: 'https://www.fifaindex.com/static/FIFA20/images/players/10/'+str(x)+'.webp')

In [270]:
photos.iloc[1235]['url_img']

'https://www.fifaindex.com/static/FIFA20/images/players/10/134779.webp'

In [271]:
data = pd.merge(players, photos, left_on='id', right_on='id', how='inner')

In [272]:
data.head()

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,style_faucon,style_buteur,style_finisseur,style_chasseur,style_cf,style_2pieds,style_magicien,style_legende,style_star,url_img
0,Alisson,1992-10-02,England,2020/2021,Liverpool,EPL,3,270,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...
1,Fabinho,1993-10-23,England,2020/2021,Liverpool,EPL,4,391,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...
2,Diogo Jota,1996-12-04,England,2020/2021,Liverpool,EPL,1,112,1,0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...
3,Thiago,1991-04-11,England,2020/2021,Liverpool,EPL,1,135,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...
4,Ederson,1993-08-17,England,2020/2021,Man City,EPL,4,360,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...


### Création d'un indicateur pépite

In [273]:
def pepite(x):
    if (x['potential'] - x['overall'] >= 6) and (x['age'] <= 23) and (x['overall'] >= 70):
        return 1
    else:
        return 0

In [274]:
data['pepite'] = data.apply(lambda x: pepite(x), axis=1)

#### suppression des doublons

In [275]:
data.to_csv('data/joueurs_avec_stats_fifa.csv', index=False)

In [276]:
players_stats_clean['Rating'].head()

0    6.82
1    6.62
2    6.85
3    6.88
4    6.79
Name: Rating, dtype: float64

In [277]:
data.head()

,Player Name,birth,nation,Season,Team,Tournament,Apps,Mins,Goals,Assists,...,style_buteur,style_finisseur,style_chasseur,style_cf,style_2pieds,style_magicien,style_legende,style_star,url_img,pepite
0,Alisson,1992-10-02,England,2020/2021,Liverpool,EPL,3,270,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...,0
1,Fabinho,1993-10-23,England,2020/2021,Liverpool,EPL,4,391,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...,0
2,Diogo Jota,1996-12-04,England,2020/2021,Liverpool,EPL,1,112,1,0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...,1
3,Thiago,1991-04-11,England,2020/2021,Liverpool,EPL,1,135,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...,0
4,Ederson,1993-08-17,England,2020/2021,Man City,EPL,4,360,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.fifaindex.com/static/FIFA20/images...,0


## Création d'un dataset style pour filtrer aisément les styles des joueurs (['id','style'])

In [278]:
joueurs = pd.read_csv('data/joueurs_avec_stats_fifa.csv')

In [279]:
style_joueurs=joueurs.set_index('id').filter(like='style_')

# style_joueurs.reset_index(inplace=True)

In [280]:
style_joueurs.head()

,style_mur,style_bouclier,style_chat,style_gant,style_sentinelle,style_garde,style_gladiateur,style_colonne_vertebrale,style_pilier,style_ombre,...,style_oeildelynx,style_faucon,style_buteur,style_finisseur,style_chasseur,style_cf,style_2pieds,style_magicien,style_legende,style_star
id,,,,,,,,,,,,,,,,,,,,,
212831,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209499,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
189509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
210257,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [281]:
style_joueurs.columns

Index(['style_mur', 'style_bouclier', 'style_chat', 'style_gant',
       'style_sentinelle', 'style_garde', 'style_gladiateur',
       'style_colonne_vertebrale', 'style_pilier', 'style_ombre',
       'style_artiste', 'style_architecte', 'style_roc', 'style_maestro',
       'style_moteur', 'style_catalyseur', 'style_tireur_delite',
       'style_oeildelynx', 'style_faucon', 'style_buteur', 'style_finisseur',
       'style_chasseur', 'style_cf', 'style_2pieds', 'style_magicien',
       'style_legende', 'style_star'],
      dtype='object')

In [282]:
style_id = pd.DataFrame(np.where(style_joueurs!=0), index=['index','style']).T

style_id['index'] = style_id['index'].apply(lambda x: style_joueurs.index[x])
style_id['style'] = style_id['style'].apply(lambda x: style_joueurs.columns[x].split('_')[1].capitalize())

In [283]:
style_id.head()

,index,style
0,212831,Mur
1,212831,Gant
2,209499,Sentinelle
3,209499,Garde
4,209499,Colonne


In [284]:
style_id['style'] = style_id['style'].apply(lambda x: x if not x =='2pieds' else '2 pieds')
style_id['style'] = style_id['style'].apply(lambda x: x if not x =='Colonne' else 'Colonne Vertébrale')
style_id['style'] = style_id['style'].apply(lambda x: x if not x =='Oeildelynx' else 'Oeil de lynx')
style_id['style'] = style_id['style'].apply(lambda x: x if not x == 'Tireur' else "Tireur d'élite")
style_id['style'] = style_id['style'].apply(lambda x: x if not x == 'Cf' else "Tireur de coups francs")
style_id['style'] = style_id['style'].apply(lambda x: x if not x == 'Ombre' else "Homme de l'ombre")

In [285]:
style_id.to_csv('data/style_id.csv', index=False)

### BROUILLON

In [286]:
a =list(set(style_id['style'].values))

In [287]:
a

['Buteur',
 "Tireur d'élite",
 'Gladiateur',
 'Roc',
 'Catalyseur',
 'Tireur de coups francs',
 'Garde',
 'Magicien',
 'Maestro',
 'Artiste',
 'Architecte',
 'Sentinelle',
 'Gant',
 'Faucon',
 "Homme de l'ombre",
 '2 pieds',
 'Star',
 'Legende',
 'Colonne Vertébrale',
 'Chasseur',
 'Pilier',
 'Moteur',
 'Finisseur',
 'Mur',
 'Oeil de lynx']

In [354]:
j = data[data['Player Name'] == 'Bernardo Silva']

In [295]:
j.index[0]

1

In [355]:
a = j.loc[j.index[0],'attacking_crossing':'defending_sliding_tackle'].sort_values(ascending=False).head(5)

In [356]:
a.index

Index(['movement_agility', 'power_stamina', 'movement_balance',
       'skill_dribbling', 'skill_ball_control'],
      dtype='object')

In [357]:
b = j.loc[j.index[0],'attacking_crossing':'defending_sliding_tackle'].sort_values().head(5)

In [362]:
b

defending_sliding_tackle      40
defending_marking             49
mentality_interceptions       49
power_jumping                 51
attacking_heading_accuracy    51
Name: 6, dtype: object

In [112]:
from googletrans import Translator
translator = Translator()
translator.translate('penaltys', src='en', dest='fr').text

'pénalités'

In [359]:
s = 'attacking_volleys skill_fk_accuracy power_long_shots movement_balance attacking_finishing'
re.sub(r'(attacking_|skill_|mentality_|power_|movement_|defending_)', '', s)

'volleys fk_accuracy long_shots balance finishing'

In [360]:
forts = [translator.translate(re.sub(r'(attacking_|skill_|mentality_|power_|movement_|defending_|_)', ' ', s), src='en', dest='fr').text.capitalize() for s in a.index]
faibles = [translator.translate(re.sub(r'(attacking_|skill_|mentality_|power_|movement_|defending_|_)', ' ', s), src='en', dest='fr').text.capitalize() for s in b.index]

In [361]:
pd.DataFrame({'Points forts': forts, 'Points faibles':faibles}).style.hide_index()

Points forts,Points faibles
Agilité,Tacle glissé
Endurance,Marquage
Équilibre,Interceptions
Dribble,Sauter
Un contrôle de balle,Précision du cap


In [259]:
data.head()

Player Name,Tournament,Club,Height,Weight,Apps,Mins,Goals,Assists,Goals/90min,Assists/90min,Yellow,Red,SpG,PS%,AerialWon,MotM,Rating,As Sub,Player_Name,Club_,Tournament_,id,birth_date,sofifa_id,player_url,short_name,long_name,age,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,nom,player_name,player_extended_name,league,date_of_birth,height,weight,poste_,poste,note_perf,perf,style,style_mur,style_bouclier,style_chat,style_gant,style_sentinelle,style_garde,style_gladiateur,style_colonne_vertebrale,style_pilier,style_ombre,style_artiste,style_architecte,style_roc,style_maestro,style_moteur,style_catalyseur,style_tireur_delite,style_oeildelynx,style_faucon,style_buteur,style_finisseur,style_chasseur,style_cf,style_2pieds,style_magicien,style_legende,style_star,url_img,pepite
Roberto Firmino,Premier League,Liverpool,181,76,34,3003,9,8,0.270000,0.240000,0,0,2.600000,79.600000,0.700000,2,7.140000,4,roberto firmino,liverpool,premier league,201942,1991-10-02,201942,https://sofifa.com/player/201942/roberto-firmino-barbosa-de-oliveira/20/159586,roberto firmino,roberto firmino barbosa de oliveira,29.000000,181,76,Brazil,liverpool,86,87,52000000,170000,"CF, ST, CAM",Right,3,4,4,High/High,Lean,Yes,100100000.000000,"#Dribbler, #Playmaker , #Engine, #Complete Midfielder",CF,9.000000,nan,06/07/2015,2023.000000,nan,nan,77.000000,82.000000,80.000000,87.000000,61.000000,78.000000,nan,nan,nan,nan,nan,nan,"Selfish, Finesse Shot, Outside Foot Shot, Crowd Favourite",72,85,78,86,81,88,80,65,76,88,78,77,80,86,81,81,81,88,74,76,73,58,87,86,72,88,60,64,42,8,11,9,6,10,83+3,83+3,83+3,83+3,84+3,84+3,84+3,83+3,84+3,84+3,84+3,83+3,82+3,82+3,82+3,83+3,72+3,73+3,73+3,73+3,72+3,69+3,67+3,67+3,67+3,69+3,roberto firmino,firmino,roberto firmino barbosa de oliveira,premier league,1991-10-02,181.000000,76.000000,CF,ATT,83.000000,4.462366,"['artiste', 'architecte', 'maestro', 'moteur', 'catalyseur', ""tireur d'élite"", 'oeil de lynx', 'faucon', 'buteur', 'finisseur', 'chasseur', '2 pieds']",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,https://www.fifaindex.com/static/FIFA20/images/players/10/201942.webp,0
Fabinho,Premier League,Liverpool,188,78,22,2075,2,3,0.090000,0.130000,6,0,0.400000,87.000000,1.500000,1,6.890000,6,fabinho,liverpool,premier league,209499,1993-10-23,209499,https://sofifa.com/player/209499/fabio-henrique-tavares/20/159586,fabinho,fabio henrique tavares,26.000000,188,78,Brazil,liverpool,85,89,45000000,125000,CDM,Right,3,2,3,Medium/High,Lean,Yes,86600000.000000,"#Tackling , #Tactician",CDM,3.000000,nan,01/07/2018,2023.000000,nan,nan,68.000000,69.000000,78.000000,77.000000,85.000000,83.000000,nan,nan,nan,nan,nan,nan,"D

In [367]:
def start_song(song_name):
    words = song_name.split()

    url = "http://www.youtube.com/results?search_query="

    for word in words:
        url += word + "+"

    time.sleep(1)   #Sleeps for a second
    webbrowser.open_new(url[:-1])  # -1 because we remove the last "+" inserted


In [371]:
import time
import webbrowser
start_song('13 block')

In [376]:
'+'.join(search_keyword.split(' '))

'Mehmet+Zobi'

In [377]:
import urllib.request
import re

search_keyword="Mehmet Zobi"
search_keyword='+'.join(search_keyword.split(' '))
html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_keyword)
video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
print("https://www.youtube.com/watch?v=" + video_ids[0])

https://www.youtube.com/watch?v=hafyLKq7jpo
